In [20]:
import pandas as pd
import numpy as np
import urllib.request
import time
from tqdm import tqdm
import os
from glob import glob
import pathlib
import zipfile

In [2]:
df = pd.read_csv('../teryt.csv',sep=';')
df

,woj_id,woj_name,pow_id,pow_name
0,2,dolnośląskie,206,karkonoski
1,2,dolnośląskie,204,górowski
2,2,dolnośląskie,207,kamiennogórski
3,2,dolnośląskie,202,dzierżonowski
4,2,dolnośląskie,205,jaworski
...,...,...,...,...
375,32,zachodniopomorskie,3208,kołobrzeski
376,32,zachodniopomorskie,3207,kamieński
377,32,zachodniopomorskie,3209,koszaliński
378,32,zachodniopomorskie,3218,łobeski


In [3]:
df['pow_id'] = np.nan_to_num(df['pow_id']).astype(int)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   woj_id    380 non-null    int64 
 1   woj_name  380 non-null    object
 2   pow_id    380 non-null    int32 
 3   pow_name  380 non-null    object
dtypes: int32(1), int64(1), object(2)
memory usage: 10.5+ KB


In [5]:
df.pow_name.unique

<bound method Series.unique of 0          karkonoski
1            górowski
2      kamiennogórski
3       dzierżonowski
4            jaworski
            ...      
375       kołobrzeski
376         kamieński
377       koszaliński
378           łobeski
379       myśliborski
Name: pow_name, Length: 380, dtype: object>

In [6]:
links = []

for bdot10kDataFormat in ['SHP', 'GML']:
    for _,row in df.iterrows():
        link = f'https://opendata.geoportal.gov.pl/bdot10k/{bdot10kDataFormat}/{row["woj_id"]:02}/{row["pow_id"]:04}_{bdot10kDataFormat}.zip'
        if bdot10kDataFormat == 'GML':
            link = f'https://opendata.geoportal.gov.pl/bdot10k/{row["woj_id"]:02}/{row["pow_id"]:04}_{bdot10kDataFormat}.zip'
        links.append(link)
        print(link)

https://opendata.geoportal.gov.pl/bdot10k/SHP/02/0206_SHP.zip
https://opendata.geoportal.gov.pl/bdot10k/SHP/02/0204_SHP.zip
https://opendata.geoportal.gov.pl/bdot10k/SHP/02/0207_SHP.zip
https://opendata.geoportal.gov.pl/bdot10k/SHP/02/0202_SHP.zip
https://opendata.geoportal.gov.pl/bdot10k/SHP/02/0205_SHP.zip
https://opendata.geoportal.gov.pl/bdot10k/SHP/02/0203_SHP.zip
https://opendata.geoportal.gov.pl/bdot10k/SHP/02/0201_SHP.zip
https://opendata.geoportal.gov.pl/bdot10k/SHP/02/0208_SHP.zip
https://opendata.geoportal.gov.pl/bdot10k/SHP/02/0215_SHP.zip
https://opendata.geoportal.gov.pl/bdot10k/SHP/02/0214_SHP.zip
https://opendata.geoportal.gov.pl/bdot10k/SHP/02/0212_SHP.zip
https://opendata.geoportal.gov.pl/bdot10k/SHP/02/0211_SHP.zip
https://opendata.geoportal.gov.pl/bdot10k/SHP/02/0213_SHP.zip
https://opendata.geoportal.gov.pl/bdot10k/SHP/02/0209_SHP.zip
https://opendata.geoportal.gov.pl/bdot10k/SHP/02/0210_SHP.zip
https://opendata.geoportal.gov.pl/bdot10k/SHP/02/0216_SHP.zip
https://

https://opendata.geoportal.gov.pl/bdot10k/10/1020_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/10/1061_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/10/1018_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/10/1063_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/10/1013_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/10/1016_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/10/1014_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/10/1015_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/10/1012_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/10/1017_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/12/1263_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/12/1219_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/12/1262_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/12/1261_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/12/1218_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/12/1204_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/12/1203_GML.zip
https://openda

https://opendata.geoportal.gov.pl/bdot10k/30/3022_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/30/3023_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/30/3021_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/30/3020_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/30/3026_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/30/3025_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/30/3019_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/30/3024_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/30/3027_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/30/3013_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/30/3014_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/30/3012_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/30/3011_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/30/3017_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/30/3016_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/30/3010_GML.zip
https://opendata.geoportal.gov.pl/bdot10k/30/3015_GML.zip
https://openda

In [23]:
pathlib.Path("data/zips").mkdir(parents=True, exist_ok=True)
pathlib.Path("data/unzipped/SHP").mkdir(parents=True, exist_ok=True)
pathlib.Path("data/unzipped/GML").mkdir(parents=True, exist_ok=True)

In [17]:
for url in tqdm(links):
    file_name = 'data/zips/' + url.split('/')[-1]
    if not os.path.exists(file_name):
        try:
            urllib.request.urlretrieve(url, file_name)
        except Exception as e:
            print(url)
            print(e)
        time.sleep(1)

100%|█████████████████████████████████████████████████████████████████████████████| 760/760 [00:00<00:00, 11676.49it/s]


In [19]:
zips = glob('data/zips/*.zip')
zips[:4]

['data/zips\\0201_GML.zip',
 'data/zips\\0201_SHP.zip',
 'data/zips\\0202_GML.zip',
 'data/zips\\0202_SHP.zip']

In [28]:
for z in tqdm(zips):
    if 'GML' in z:
        out_dir = 'data/unzipped/GML'
    else:
        out_dir = 'data/unzipped/SHP'
    with zipfile.ZipFile(z, 'r') as zip_ref:
        zip_ref.extractall(out_dir)

 26%|█████████████████████▏                                                          | 201/760 [17:32<48:47,  5.24s/it]


OSError: [Errno 28] No space left on device